# CS202: Compiler Construction

## In-class Exercises, Week of 01/31/2022

----
## Questions from last week

## Question 10

Convert the following program into A-normal form.

In [45]:
python = """
x = 1 + 2 + 3
"""

In [46]:
python_anf = """
tmp1 = 2 + 3
x = 1 + tmp1
"""

## Question 11

Describe a recursive procedure to perform the *remove-complex-opera* pass. Reference section 2.4 in the textbook.

I already did this so I'm just going to paste my code here.

I did things slightly differently from how the book describes. In the book, rco_exp returns type Tuple[expr, list[Tuple(str, expr)]], but i found it easier to generate the statements within rco_exp and return them directly. It also doesn't describe a mechanism by which to tell expressions that are top level and therefore do not need to be broken down from expressions that are within other expressions and do need to be broken down. I did this by adding a parameter "top" to rco_exp which is passed True when called from rco_stmt and False when called from rco_exp. If there's a more elegant solution I couldn't think of it.

rco_stmt(statement: stmt) -> list[stmt]:
call rco_exp on the subexpression of Expr or Assign with toplevel designated true and return those statements along with the original statement modified to contain the result of rco_exp.

rco_exp(exp: expr, top: bool) -> Tuple[list[stmt], expr]:
if constant or variable name, return empty list and the atom.
otherwise, call rco_exp on all subexpressions with toplevel designated false, replace the subexpressions in exp with the return of rco_exp and add the lists of statements to this call's list of statements.
if toplevel is false, then add a temporary assignment to the list of statements and return that with the temporary variable name substituted for exp.

call rco_stmt on every statement and return a module with all the lists strung together.


def rco(prog: Module) -> Module:
    """
    Removes complex operands. After this pass, the arguments to operators (unary and binary
    operators, and function calls like "print") will be atomic.
    :param prog: An Lvar program
    :return: An Lvar program with atomic operator arguments.
    """

    def rco_exp(exp: expr, top: bool) -> Tuple[list[stmt], expr]:
        statements = []
        match exp:
            case Constant(_) | Name(_):
                return [], exp
            case Call(Name(n), args):
                new_args = []
                for arg in args:
                    new_statements, new_arg = rco_exp(arg, False)
                    new_args.append(new_arg)
                    statements.extend(new_statements)
                exp = Call(Name(n), new_args)
            case UnaryOp(op, i):
                statements, i = rco_exp(i, False)
                exp = UnaryOp(op, i)
            case BinOp(left, op, right):
                statements, left = rco_exp(left, False)
                statements2, right = rco_exp(right, False)
                statements.extend(statements2)
                exp = BinOp(left, op, right)
            case _:
                raise Exception("rco/rco_exp")

        if not top:
            tmp = Name(gensym("(tmp)")) # use parentheses because they are not allowed in normal variable names.
            statements.append(Assign([tmp], exp))
            exp = tmp
        return statements, exp

    def rco_stmt(statement: stmt) -> list[stmt]:
        match statement:
            case Expr(exp):
                new_statements, exp = rco_exp(exp, True)
                new_statements.append(Expr(exp))
            case Assign([var], exp):
                new_statements, exp = rco_exp(exp, True)
                new_statements.append(Assign([var], exp))
            case _:
                raise Exception("rco")
        return new_statements

    new_prog = Module([])
    for statement in prog.body:
        new_prog.body.extend(rco_stmt(statement))

    return new_prog

----
# Select-instructions pass

The select-instructions pass transforms a sequence of statements into X86 assembly instructions.

## Question 1

Convert the following `Lvar` code into a psuedo-x86 assembly program.

```
Module([Assign([Name("y")], Constant(5)),
        Assign([Name("x")], Name("y")),
        Expr(Call(Name("print"), [Name("x")]))
])
```

Program(
 {
  'start':
   [
    NamedInstr(
     "movq",
     [
      Immediate(5),
      Var("y")
     ]),
    NamedInstr(
     "movq",
     [
      Var("y"),
      Reg("rax")
     ]),
    NamedInstr(
     "movq",
     [
      Reg("rax"),
      Var("x")
     ]),
    NamedInstr(
     "movq",
     [
      Var("x"),
      Reg("rdi")
     ]),
    Callq("print_int"),
    Jmp("conclusion")
   ]
 })

## Question 2

Describe the structure of select-instructions.

select_instructions knows that it's being passed a list of atomic expressions. So it only needs to translate between all the possibilities for an atomic expression and the x86 instructions which correspond.

atom_to_arg(exp: expr) -> x86.Arg:
turns constants into immediates, and named variables into x86var Vars

def translate_expression(expression: expr) -> list[x86.Instr]:
matches all possibilities with the required x86 to put the correct return value of the expr into rax.

translate_statement(statement: stmt) -> list[x86.Instr]:
if stmt is Expr, return translate_expression of the expr inside.
if stmt is Assign, add an instruction to move rax into the variable name

for every statement, call translate_statement and return all the instructions strung together.


def select_instructions(prog: Module) -> x86.Program:
    """
    Transforms a Lvar program into a pseudo-x86 assembly program.
    :param prog: a Lvar program
    :return: a pseudo-x86 program
    """

    valid_calls = {
        'print': {
            'name': 'print_int',
            'args': 1,
        },
        'input_int': {
            'name': 'read_int',
            'args': 0,
        }
    }

    def atom_to_arg(exp: expr) -> x86.Arg:
        match exp:
            case Constant(i):
                return x86.Immediate(i)
            case Name(n):
                return x86.Var(n)
            case _:
                raise Exception("select_instructions/atom_to_arg")

    def translate_expression(expression: expr) -> list[x86.Instr]:

        def move_to_rax(exp: expr) -> x86.Instr:
            return x86.NamedInstr("movq",
                                  [atom_to_arg(exp), x86.Reg("rax")]
                                  )

        instructions = []
        match expression:
            case Call(Name(name), args):
                if name not in valid_calls.keys() or \
                        len(args) != valid_calls[name]['args']:
                    raise Exception("select_instructions/call_builtin")
                if len(args) == 1:
                    instructions.append(x86.NamedInstr("movq", [atom_to_arg(args[0]), x86.Reg("rdi")]))
                instructions.append(x86.Callq(valid_calls[name]["name"]))
                return instructions
            case UnaryOp(op, exp):
                match op:
                    case USub():
                        instructions = [
                            move_to_rax(exp),
                            x86.NamedInstr("negq",
                                           [x86.Reg("rax")]
                                           )
                        ]
                    case _:
                        raise Exception('select_instructions/unary_operation')
            case BinOp(left, op, right):
                match op:
                    case Add():
                        instr_name = "addq"
                    case Sub():
                        instr_name = "subq"
                    case _:
                        raise Exception('select_instructions/binary_operation')
                return [
                    move_to_rax(left),
                    x86.NamedInstr(instr_name,
                                   [atom_to_arg(right), x86.Reg("rax")]
                                   )
                ]
            case Constant(_) | Name(_):
                return [
                    move_to_rax(expression)
                ]
            case _:
                raise Exception("select_instructions")

    def translate_statement(statement: stmt) -> list[x86.Instr]:
        match statement:
            case Expr(exp):
                return translate_expression(exp)
            case Assign([Name(n)], exp):
                instructions = translate_expression(exp)
                match instructions:
                    case [x86.NamedInstr("movq", [x86.Immediate(i), x86.Reg("rax")])]:
                        return [x86.NamedInstr("movq",
                                               [x86.Immediate(i), atom_to_arg(Name(n))]
                                               )
                                ]
                    case _:
                        instructions.append(
                            x86.NamedInstr("movq",
                                           [x86.Reg("rax"), atom_to_arg(Name(n))]
                                           )
                        )
                        return instructions
            case _:
                raise Exception("select_instructions/translate_statement")

    instructions : list[x86.Instr] = []
    for statement in prog.body:
        instructions.extend(translate_statement(statement))
    instructions.append(x86.Jmp("conclusion"))

    return x86.Program({
        "start": instructions
    })


----
# Assign-homes pass

The assign-homes pass places each program variable in a *stack location* in memory, eliminating variables from the program.

See Section 2.2 for details; especially see Figure 2.8 for details on the memory layout of stack frames.

## Question 3

Write X86 assembly that prepares a stack frame for four variables and puts the values 1,2,3,4 in stack locations.

In [3]:
from cs202_support.eval_x86 import X86Emulator

asm = """
    pushq %rbp
    movq %rsp, %rbp
    subq $32, %rsp
    movq $1, -8(%rbp)
    movq $2, -16(%rbp)
    movq $3, -24(%rbp)
    movq $4, -32(%rbp)

"""

X86Emulator(logging=False).eval_instructions(asm)

,Location,Old,New
0,mem 992,NaN,1000
1,mem 984,NaN,1
2,mem 976,NaN,2
3,mem 968,NaN,3
4,mem 960,NaN,4
5,reg rbp,1000.0,992
6,reg rsp,1000.0,960


## Question 4

Write X86 assembly that prepares a stack frame for three variables and puts the values 1,2,3 in stack locations. Why is this situation different than above?

In [6]:
asm = """
pushq %rbp
    movq %rsp, %rbp
    subq $24, %rsp
    movq $1, -8(%rbp)
    movq $2, -16(%rbp)
    movq $3, -24(%rbp)
"""

X86Emulator(logging=False).eval_instructions(asm)

,Location,Old,New
0,mem 992,NaN,1000
1,mem 984,NaN,1
2,mem 976,NaN,2
3,mem 968,NaN,3
4,reg rbp,1000.0,992
5,reg rsp,1000.0,968


YOUR SOLUTION HERE
We're violating x86 standardization by allocating memory not in multiples of 16 bytes.

## Question 5

Implement a function `align` to ensure 16-byte alignment.

In [7]:
def align(num_bytes: int) -> int:
    return num_bytes if num_bytes % 16 == 0 else num_bytes + 8

print(align(32))
print(align(8))
print(align(24))

32
16
32


## Question 6

Describe the assign-homes pass.

YOUR SOLUTION HERE

global variable vars contains a mapping from string variable names to offsets, as well as the number of variables allocated.

replace_var(var: x86.Arg) -> x86.Arg:
    if not a variable, return the input.
    if var name is in vars, return an x86 deref of the stack pointer offset by the value in vars,
    else, add one to vars["(num_vars)"], add the var name with that * 8 to vars, and return  an x86 deref.

replace_instr(instruction: x86.Instr) -> x86.Instr:
    for all the arguments of the instruction, replace them with replace_var and return the modified instruction

run replace instruction on every instruction, return the modified list of instructions along with vars["(num_vars)"] * 8 16-byte aligned.



def assign_homes(program: x86.Program) -> Tuple[x86.Program, int]:
    """
    Assigns homes to variables in the input program. Allocates a stack location for each
    variable in the program.
    :param program: An x86 program.
    :return: A Tuple. The first element is an x86 program (with no variable references).
    The second element is the number of bytes needed in stack locations.
    """
    vars = {
        "(num_vars)": 0
    }

    def replace_var(var: x86.Arg) -> x86.Arg:
        match var:
            case x86.Var(name):
                if name not in vars.keys():
                    vars["(num_vars)"] += 1
                    vars[name] = vars["(num_vars)"] * -8
                return x86.Deref("rbp", vars[name])
            case _:
                return var

    def replace_instr(instruction: x86.Instr) -> x86.Instr:
        match instruction:
            case x86.NamedInstr(name, args):
                return x86.NamedInstr(name, [replace_var(arg) for arg in args])
            case _:
                return instruction

    new_instructions = []
    match program:
        case x86.Program({"start": instructions}):
            for instruction in instructions:
                new_instructions.append(replace_instr(instruction))

    def calc_stack(num_vars: int) -> int:
        return (num_vars if num_vars % 2 == 0 else num_vars + 1) * 8

    return x86.Program({
        "start": new_instructions
    }), calc_stack(vars["(num_vars)"])



----
# Patch-instructions pass

The patch-instructions pass fixes instructions with two in-memory arguments, by using the `rax` register as a temporary location.

## Question 7

What is wrong with the following instructions?

```
movq -8(%rbp), -16(%rbp)
addq -24(%rbp), -16(%rbp)
```

YOUR SOLUTION HERE
it's illegal in x86 to move memory from the stack directly to the stack.

## Question 8

Fix the instructions above.

movq -8(%rbp), %rax
addq -24(%rbp), %rax
movq %rax, -16(%rbp)

## Question 9

Describe the patch-instructions pass.

patch_instruction(instruction: x86.Instr) -> [x86.Instr]:
    if the instruction is a NamedInstr with two derefs from the stack as arguments,
    first move the left deref into rax, then perform the namedinstr on rax with the right deref, then move the result back into the left deref.

return the results of patch_instruction for every instruction.

I think the way I set up select_instructions this pass isn't needed, so I'm not 100% confident it works

def patch_instructions(inputs: Tuple[x86.Program, int]) -> Tuple[x86.Program, int]:
    """
    Patches instructions with two memory location inputs, using %rax as a temporary location.
    :param inputs: A Tuple. The first element is an x86 program. The second element is the
    stack space in bytes.
    :return: A Tuple. The first element is the patched x86 program. The second element is
    the stack space in bytes.
    """

    def patch_instruction(instruction: x86.Instr) -> [x86.Instr]:
        match instruction:
            case x86.NamedInstr(name, [x86.Deref("rbp", off1), x86.Deref("rbp", off2)]):
                return [
                    x86.NamedInstr("movq", [x86.Deref("rbp", off1), x86.Reg("rax")]),
                    x86.NamedInstr(name, [x86.Reg("rax"), x86.Deref("rbp", off2)]),
                    x86.NamedInstr("movq", [x86.Reg("rax"), x86.Deref("rbp", off1)])
                ]
            case _:
                return [instruction]

    new_instructions = []
    match inputs[0]:
        case x86.Program({"start": instructions}):
            for instruction in instructions:
                new_instructions.extend(patch_instruction(instruction))

    return x86.Program({
        "start": new_instructions
    }), inputs[1]